# Diet problem
(Adapted from the pyomo/examples/pyomo files)

The goal of this problem is to *minimize* cost.  
The variables of the problem include the decision of weather a certain food is bought or not.  
Furthermore, the constraints ensure that *at least* one entree, side and drink are selected.

This model is written with the following structure:  

1. Package imports
2. Model and Set declaration
3. Parameter declaration and initialization
4. Variable declaration and initialization
5. Constraint declaration and initialization
6. Objective function
7. Solution

### 1. Package imports

In [1]:
from pyomo.environ import *   #: Same as usual
from pyomo.opt import SolverFactory

### 2. Model and Set declaration

Note how the model *must* be declared first. Subsequently, every component from the model is attached to the model.

In [28]:
m = ConcreteModel()  #: Declare model

In [4]:
foods = ["QPwCheese", "MDwCheese", "LeBigMac", "FOFish", "McGChicken", "Fries", "McSausage", "LfMilk", "OJ"]
entree = ["QPwCheese", "MDwCheese", "LeBigMac", "FOFish", "McGChicken"]
side = ["Fries", "McSausage"]
drink = ["LfMilk", "OJ"]

In [5]:
m.food = Set(initialize=foods)

### 3. Parameter declaration and initialization

In [6]:
COST_DICT = dict.fromkeys(foods)

In [7]:
COST_DICT = {"QPwCheese": 1.84,
             "MDwCheese": 2.19, 
             "LeBigMac": 1.84, 
             "FOFish": 1.44, 
             "McGChicken": 2.29, 
             "Fries": 0.77, 
             "McSausage": 1.29, 
             "LfMilk": 0.60, 
             "OJ": 0.72}

In [8]:
m.cost = Param(m.food, initialize=COST_DICT, within=PositiveReals)

In [9]:
m.f_min = Param(m.food, within=NonNegativeReals, default=0.0)

In [10]:
MAX_FOOD_SUPPLY = 20.0

In [11]:
def f_max_validate (model, value, j):
    return m.f_max[j] > m.f_min[j]

In [12]:
m.f_max = Param(m.food, validate=f_max_validate, default=MAX_FOOD_SUPPLY)

In [13]:
NUTR = ["Cal", "Carbo", "Protein", "VitA", "VitC", "Calc", "Iron"]

In [14]:
m.nutr = Set(initialize=NUTR)

In [15]:
N_MIN = {"Cal": 2000, "Carbo": 350, "Protein": 55, "VitA": 100, "VitC": 100, "Calc": 100, "Iron": 100}

In [16]:
m.n_min = Param(m.nutr, initialize=N_MIN)

In [17]:
N_MAX = {"Cal": 0, "Carbo": 375, "Protein": 0, "VitA": 0, "VitC": 0, "Calc": 0, "Iron": 0}

In [18]:
m.n_max = Param(m.nutr, initialize=N_MAX)

In [19]:
AMT = {}
AMT[("QPwCheese","Cal")] = 510
AMT[("MDwCheese","Cal")] = 370
AMT[("LeBigMac","Cal")] = 500
AMT[("FOFish","Cal")] = 370
AMT[("McGChicken","Cal")] = 400
AMT[("Fries","Cal")] = 220
AMT[("McSausage","Cal")] = 345
AMT[("LfMilk","Cal")] = 110
AMT[("OJ","Cal")] = 80
AMT[("QPwCheese","Carbo")] = 34
AMT[("MDwCheese","Carbo")] = 35
AMT[("LeBigMac","Carbo")] = 42
AMT[("FOFish","Carbo")] = 38
AMT[("McGChicken","Carbo")] = 42
AMT[("Fries","Carbo")] = 26
AMT[("McSausage","Carbo")] = 27
AMT[("LfMilk","Carbo")] = 12
AMT[("OJ","Carbo")] = 20
AMT[("QPwCheese","Protein")] = 28
AMT[("MDwCheese","Protein")] = 24
AMT[("LeBigMac","Protein")] = 25
AMT[("FOFish","Protein")] = 14
AMT[("McGChicken","Protein")] = 31
AMT[("Fries","Protein")] = 3
AMT[("McSausage","Protein")] = 15
AMT[("LfMilk","Protein")] = 9
AMT[("OJ","Protein")] = 1
AMT[("QPwCheese","VitA")] = 15
AMT[("MDwCheese","VitA")] = 15
AMT[("LeBigMac","VitA")] = 6
AMT[("FOFish","VitA")] = 2
AMT[("McGChicken","VitA")] = 8
AMT[("Fries","VitA")] = 0
AMT[("McSausage","VitA")] = 4
AMT[("LfMilk","VitA")] = 10
AMT[("OJ","VitA")] = 2
AMT[("QPwCheese","VitC")] = 6
AMT[("MDwCheese","VitC")] = 10
AMT[("LeBigMac","VitC")] = 2
AMT[("FOFish","VitC")] = 0
AMT[("McGChicken","VitC")] = 15
AMT[("Fries","VitC")] = 15
AMT[("McSausage","VitC")] = 0
AMT[("LfMilk","VitC")] = 4
AMT[("OJ","VitC")] = 120
AMT[("QPwCheese","Calc")] = 30
AMT[("MDwCheese","Calc")] = 20
AMT[("LeBigMac","Calc")] = 25
AMT[("FOFish","Calc")] = 15
AMT[("McGChicken","Calc")] = 15
AMT[("Fries","Calc")] = 0
AMT[("McSausage","Calc")] = 20
AMT[("LfMilk","Calc")] = 30
AMT[("OJ","Calc")] = 2
AMT[("QPwCheese","Iron")] = 20
AMT[("MDwCheese","Iron")] = 20
AMT[("LeBigMac","Iron")] = 20
AMT[("FOFish","Iron")] = 10
AMT[("McGChicken","Iron")] = 8
AMT[("Fries","Iron")] = 2
AMT[("McSausage","Iron")] = 15
AMT[("LfMilk","Iron")] = 0
AMT[("OJ","Iron")] = 2

In [20]:
m.amt = Param(m.food, m.nutr, initialize=AMT)

In [21]:
def buy_bounds(mod, i):
    return (mod.f_min[i], mod.f_max[i])
m.buy = Var(m.food, bounds=buy_bounds, within=NonNegativeIntegers)

In [22]:
def total_cost_rule(mod):
    return sum(mod.cost[j] * mod.buy[j] for j in mod.food)
m.total_cost = Objective(rule=total_cost_rule, sense=minimize)

In [23]:
def entree_rule(mod):
    return sum(mod.buy[e] for e in entree) >= 1
m.entree = Constraint(rule=entree_rule)

In [24]:
def side_rule(mod):
    return sum(mod.buy[s] for s in side) >= 1
m.side = Constraint(rule=side_rule)

In [25]:
def drink_rule(mod):
    return sum(mod.buy[d] for d in drink) >=1
m.drink = Constraint(rule=drink_rule)

In [26]:
opt = SolverFactory('cbc')  #: declare the solver

In [27]:
results = opt.solve(m, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Aug 21 2019 

command line - /home/dav0/in_dev_/test_pynumero/miniconda/envs/kslt/bin/cbc -printingOptions all -import /tmp/tmpblflt4k5.pyomo.lp -stat=1 -solve -solu /tmp/tmpblflt4k5.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 3 (-1) rows, 9 (-1) columns and 9 (-1) elements
Statistics for presolved model
Original problem has 9 integers (0 of which binary)
Presolved problem has 9 integers (0 of which binary)
==== 0 zero objective 8 different
1 variables have objective of 0.6
1 variables have objective of 0.72
1 variables have objective of 0.77
1 variables have objective of 1.29
1 variables have objective of 1.44
2 variables have objective of 1.84
1 variables have objective of 2.19
1 variables have objective of 2.29
==== absolute objective values 8 different
1 variables have objective of 0.6
1 variables have objective of 0.72
1 variables have objective of 0.77
1 variables have 